In [1]:
import pandas as pd
import numpy as np
import math as m
import DataHandling

In [2]:
roadId = input('Please write the Road ID: ')

Please write the Road ID: N1


In [3]:
roads = DataHandling.readRoads(roadId)
bridges = DataHandling.readBridges(roadId)
widths = DataHandling.readWidths(roadId)
traffic = DataHandling.readTraffic(roadId)

In [4]:
def formatWidthData(dataframe = widths):
    dataframe = dataframe.drop('width', 1)
    dataframe = dataframe.drop('roadId', 1)
    dataframe = dataframe.sort_values(by='startChainage').reset_index()
    dataframe = dataframe.drop('index', 1)
    return dataframe

In [5]:
def formatBridgeData(dataframe = bridges):
    dataframenoduplicates = dataframe.drop_duplicates('LRPName', keep = 'first')
    dataframemodified = dataframenoduplicates.ix[:,:10]
    dataframemodified = dataframemodified.drop('km', 1)
    dataframemodified = dataframemodified.drop('structureNr', 1)
    dataframemodified = dataframemodified.drop('type', 1)
    dataframemodified = dataframemodified.drop('roadName', 1)
    dataframemodified = dataframemodified.drop('length', 1)
    dataframemodified = dataframemodified.drop('name', 1)
    dataframemodified = dataframemodified.sort_values(by='chainage').reset_index()
    dataframemodified = dataframemodified.drop('index', 1)
    return dataframemodified

In [6]:
def formatTrafficData(dataframe = traffic):
    dataframe = dataframe.drop('linkNo',1)
    dataframe = dataframe.drop('name',1)
    dataframe = dataframe.drop('start_lrp',1)
    dataframe = dataframe.drop('start_offset',1)
    dataframe = dataframe.drop('end_lrp',1)
    dataframe = dataframe.drop('end_offset',1)
    dataframe = dataframe.drop('length',1)
    dataframe = dataframe.reset_index()
    dataframe = dataframe.drop('index',1)
    dataframe.columns = dataframe.columns.str.replace('start_chainage','startChainage')
    dataframe.columns = dataframe.columns.str.replace('end_chainage','endChainage')
    return dataframe


In [7]:
def mergeFiles(trafficData=formatTrafficData(), bridgeData = formatBridgeData(), widthData = formatWidthData()):
    indexesMergedData = widthData['startChainage'].searchsorted(bridgeData['chainage'], side = 'right')
    indexesMergedData =indexesMergedData.tolist()
    addedNrLanes = bridgeData
    list_a = []
    for i in indexesMergedData:
        list_a.append(widthData['nrLanes'][int(i)-1])
    addedNrLanes['nrLanes']=list_a
    bridgeData = addedNrLanes
    addedTrafficData = bridgeData
    for title in trafficData.ix[:,3:].columns:
        indexesMergedData = trafficData['startChainage'].searchsorted(bridgeData['chainage'], side = 'right')
        indexesMergedData =indexesMergedData.tolist()
        list_b=[]
        for i in indexesMergedData:
            list_b.append(trafficData[title][int(i)-1])
        addedTrafficData[title]=list_b
    return addedTrafficData

In [8]:
def calculateLaneTrafficDensity(dataframe = mergeFiles()):
    for title in dataframe.ix[:,5:].columns:
        dataframe[title] = dataframe[title]/dataframe['nrLanes']
    return dataframe

In [9]:
#Truck capacity is max capacity of type of trucks in tonnes
def calcualteVulnerability(dataframe = calculateLaneTrafficDensity(), truckcapacity = [15,10,5], conditionlikelihood = [0.05,0.10,0.15,0.20]):
    vulbasematrix = dataframe.ix[:,:4]
    vulbasematrix.loc[vulbasematrix.condition == 'A', 'BridgeFailureLikelihood'] = conditionlikelihood[0]
    vulbasematrix.loc[vulbasematrix.condition == 'B', 'BridgeFailureLikelihood'] = conditionlikelihood[1]
    vulbasematrix.loc[vulbasematrix.condition == 'C', 'BridgeFailureLikelihood'] = conditionlikelihood[2]
    vulbasematrix.loc[vulbasematrix.condition == 'D', 'BridgeFailureLikelihood'] = conditionlikelihood[3]
    cargotitles = ['heavyTruck', 'mediumTruck', 'smallTruck']
    vulbasematrix['TrafficEconomicVulnerability'] = dataframe[cargotitles[0]]*truckcapacity[0]+dataframe[cargotitles[1]]*truckcapacity[1]+dataframe[cargotitles[2]]*truckcapacity[2]
    vulbasematrix['TotalEconomicVulnerability'] = vulbasematrix['TrafficEconomicVulnerability']*vulbasematrix['BridgeFailureLikelihood']
    return vulbasematrix

In [13]:
calcualteVulnerability()

,road,LRPName,condition,chainage,BridgeFailureLikelihood,TrafficEconomicVulnerability,TotalEconomicVulnerability
0,N1,LRP001a,A,1.8,0.05,14037.000000,701.850000
1,N1,LRP004b,A,4.925,0.05,7396.000000,369.800000
2,N1,LRP008b,A,8.976,0.05,15647.500000,782.375000
3,N1,LRP010a,B,10.543,0.10,15647.500000,1564.750000
4,N1,LRP010b,A,10.88,0.05,15647.500000,782.375000
5,N1,LRP010c,A,10.897,0.05,15647.500000,782.375000
6,N1,LRP011a,A,11.296,0.05,15647.500000,782.375000
7,N1,LRP011c,B,11.808,0.10,7823.750000,782.375000
8,N1,LRP012a,A,12.239,0.05,23216.666667,1160.833333
9,N1,LRP012b,A,12.253,0.05,23216.666667,1160.833333


In [16]:
def printOneRoadResults(dataframe = calcualteVulnerability(), roadId = roadId):
    dataframe.to_csv('vulnerabilityroad'+roadId+'.csv')